In [9]:
using CSV
using DataFrames
using LinearAlgebra
using SparseArrays
using Random
Random.seed!(1234);  # determinism

In [10]:
# Read network data
I, J = Vector{Int64}(), Vector{Int64}()
open("Florida-bay.txt") do f
    for line in eachline(f)
        i, j = [parse(Int64, v) for v in split(line)]
        if i != j
            push!(I, i + 1); push!(J, j + 1);  # 1-index
        end
    end
end
n = max(maximum(I), maximum(J)) # number of nodes
A = sparse(I, J, 1, n, n)       # adjacency matrix (transpose)

# Read names
df = CSV.File("Florida-bay-meta.csv") |> DataFrame
names = df[:, :name];

In [11]:
# normalize columns to form column sub-stochastic matrix P
At = A'
P = zeros(n, n)
for j = 1:n
    if sum(At[:, j]) > 0
        P[:, j] = At[:, j] / sum(At[:, j])
    end
end
v = ones(Float64, n) / n  # uniform teleportation
α = 0.85;

In [14]:
# Basic PageRank iteration
xk = copy(v)
for k = 1:200
    xk = α * P * xk + (1 - α) * v
    xk = xk / sum(xk)
end
xk

sp = sortperm(xk, rev=true)
names[sp[1:10]]

10-element Array{Union{Missing, String},1}:
 "Respiration"              
 "Water POC"                
 "Raptors"                  
 "Output"                   
 "Crocodiles"               
 "Benthic POC"              
 "Suspension Feeding Polych"
 "Dolphin"                  
 "Acartia Tonsa"            
 "Oithona nana"             